In [1]:
import torch.nn
from datetime import datetime
import torchvision.models as models
import torch.nn.functional as F
import time
import torchvision.models as models
from torch import nn, optim
from tqdm import tqdm
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import random

In [2]:
def get_dataloaders(dataset, train_ratio, val_ratio, batch_size):
    train_dataset = dataset
    val_dataset = dataset
    test_dataset = dataset
    # obtain training indices that will be used for validation
    num_train = len(test_dataset)
    indices = list(range(num_train))
    print("--------- INDEX checking ---------")
    print(f"Original: {indices[:5]}")
    random.shuffle(indices)
    print(f"Shuffled: {indices[:5]}")
    print("--------- INDEX shuffled ---------\n")

    split_train = int(np.floor(train_ratio * num_train))
    split_val = split_train + int(np.floor(val_ratio * (num_train-split_train)))
    train_idx, val_idx, test_idx = indices[0:split_train], indices[split_train:split_val], indices[split_val:]
    merge_dataset = Subset(train_dataset, train_idx)

    train_loader = DataLoader(merge_dataset, batch_size=batch_size)
    val_loader = DataLoader(Subset(val_dataset, val_idx), batch_size=batch_size)
    test_loader = DataLoader(Subset(test_dataset, test_idx), batch_size=batch_size)
    
    # check dataset
    print(f"Total number of samples: {num_train} datapoints")
    print(f"Number of train samples: {len(train_loader)} batches/ {len(train_loader.dataset)} datapoints")
    print(f"Number of val samples: {len(val_loader)} batches/ {len(val_loader.dataset)} datapoints")
    print(f"Number of test samples: {len(test_loader)} batches/ {len(test_loader.dataset)} datapoints")
    print(f"")
    
    dataloaders = {
        "train": train_loader,
        "val": val_loader,
        "test": test_loader,
    }
    return dataloaders

In [3]:
def load_original_data_pytorch(path, train_ratio, val_ratio, batchsize):
    # Define a transform to normalize the data
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])
    
    # Download and load the training data
    trainset = datasets.MNIST(path, download=True, train=True, transform=transform)
    # train_loader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)
    dataloaders = get_dataloaders(trainset,train_ratio, val_ratio, batchsize)
    # Download and load the test data
    testset = datasets.MNIST(path, download=True, train=False, transform=transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=True)

    loaders = {
    'train': dataloaders['train'],
    'val': dataloaders['val'],
    'test': test_loader,
    }
    return loaders


In [4]:
def pprint(output = '\n', show_time = False): # print and fprint at the same time
    filename = "hw2-1-MAR27.txt"
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
pprint("build function", True)

build function


In [5]:
def count_parameters(model):
    total_num = 0

    for parameter in model.parameters():
        if parameter.requires_grad:
            total_num += parameter.numel() 
    return total_num


In [6]:

def train(model, model_name, loaders, optim_op, lr):
    model = model()
    loaders = loaders()
    pprint(f"test {model_name}", True)
    model_parameters_amount = count_parameters(model)
    pprint(f"model total parameters: {model_parameters_amount:,}")

    model = model.cuda()
    criterion = nn.CrossEntropyLoss()
    lr= 0.005
    if optim_op == 0:
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optim_op == 1:
        optimizer = optim.SGD(model.parameters(), lr=lr)
    pprint(f"learning rate={lr}")
    iteration = 0
    epochs = 20
    start = time.time()
    phases = ['train', 'val']
    for epoch in range(epochs):
        for phase in phases:
            running_loss = 0.0
            correct_predictions = 0
            correct_top3_predictions = 0
            total_samples = 0
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            for images, labels in tqdm(loaders[phase]): # Iterate over data.
                images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                loss = criterion(outputs, labels)

                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train': # backward + optimize only if in training phase
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item()

                # Convert outputs to predicted class by selecting the class with the highest score
                _, predicted = torch.max(outputs, 1)
                # Accumulate the number of correct predictions
                correct_predictions += (predicted == labels).sum().item()
                
                _, top3_preds = outputs.topk(3, 1, True, True)
                correct_top3_predictions += sum([labels[i] in top3_preds[i] for i in range(labels.size(0))])

                total_samples += labels.size(0)
                iteration += 1
                # if iteration % 20 == 0:
                #     print(iteration)
            avg_loss = running_loss / total_samples
            top1_accuracy = correct_predictions / total_samples * 100
            top3_accuracy = correct_top3_predictions / total_samples * 100
            pprint(f"Epoch [{epoch+1}/{epochs}], phase: {phase}, samples: {total_samples}, Loss: {avg_loss:.4f}, Top-1 Accuracy: {top1_accuracy:.2f}%, Top-3 Accuracy: {top3_accuracy:.2f}%")
    end = time.time()
    duration = end - start
    pprint(f"Elapsed time: {duration} seconds")
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save(f'{model_name}.pt') # Save
    pprint(f"weight saved as: {model_name}.pt")   


In [11]:
class SimpleNN(nn.Module):
    def __init__(self, act_layer):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 10)
        if act_layer == "softmax":
            self.activation = F.softmax
        elif act_layer == "sigmoid":
            self.activation = F.sigmoid
        elif act_layer == "ReLU":
            self.activation = F.relu
        elif act_layer == "leakyReLU":
            self.activation = F.leaky_relu
    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 64)  # The image size is reduced to 7x7 after pooling layers
        self.fc2 = nn.Linear(64, 10)  # 10 output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Convolution -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Convolution -> ReLU -> Pooling
        x = torch.flatten(x, 1)  # Flatten
        x = F.relu(self.fc1(x))  # Dense layer -> ReLU
        x = self.fc2(x)  # Output layer
        return F.log_softmax(x, dim=1)  # Log Softmax activation for the output


In [12]:
path = 'D:\\Casper\\OTHER\\Data\\MNIST_data'
model_list = [
    lambda: SimpleNN("leakyReLU"),
    # SimpleCNN(),
]

model_name = [
    "SimpleNN",
    "SimpleCNN",
]
loaders = [
    lambda: load_original_data_pytorch(path, 0.2, 0.2, 4),
    lambda: load_original_data_pytorch(path, 0.2, 0.2, 64),
    lambda: load_original_data_pytorch(path, 0.2, 0.2, 1024),
]
lr = [
    0.001,
    0.005,
    0.025,
]
optimizers = [
    0,
    1
]
for ii in range(len(model_name)):
    train(model_list[ii], model_name[ii], loaders[ii], optimizers[ii], lr[ii])

--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [46871, 58840, 10034, 17816, 45627]
--------- INDEX shuffled ---------

Total number of samples: 60000 datapoints
Number of train samples: 3000 batches/ 12000 datapoints
Number of val samples: 2400 batches/ 9600 datapoints
Number of test samples: 9600 batches/ 38400 datapoints

test SimpleNN
model total parameters: 50,890
learning rate=0.005


  1%|▏         | 38/3000 [00:00<00:08, 345.57it/s]

100%|██████████| 3000/3000 [00:07<00:00, 379.94it/s]


Epoch [1/20], phase: train, samples: 12000, Loss: 0.1660, Top-1 Accuracy: 79.20%, Top-3 Accuracy: 94.51%


100%|██████████| 2400/2400 [00:03<00:00, 649.67it/s]


Epoch [1/20], phase: val, samples: 9600, Loss: 0.1287, Top-1 Accuracy: 85.44%, Top-3 Accuracy: 96.39%


 20%|█▉        | 596/3000 [00:01<00:06, 370.31it/s]


KeyboardInterrupt: 